#Assignment description

**Prerequisites**:

1. Authorize with your Google account
2. Go to Google Cloud Console
3. Select existing Google Cloud project or create a new one
4. Enable Google BigQuery API for the selected project
5. Copy this document to your google drive

**Implementation**:
1. Complete all assignments in this document
2. Share your variant of the document with us

In [ ]:
#run this cell first
import numpy as np
import pandas as pd
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


#Assignment #1
Prepare SQL which converts raw hits (user events) into sessions.

Count the number of sessions in the result table.

**In**:
 - [Session definition](https://support.google.com/analytics/answer/2731565?hl=en)
 - Public table with samle hits data: `analytics-230012.Assignments.hits_for_sessions`
 - Use your Google Cloud project ID instead of  `project_id`

**Комментарии:**


---


- Учтено: полчаса неактивности с возобновлением хитов на сайте; полночь (но в примере, если я не путаю, таких кейсов не было); изменение utm_campaign
- Было несколько решений, они давали близкие друг другу значения, но все же отличные друг от друга.
- Решил оставить тот, который буквально следуют правилам GA + в отдельной колонке отмечает переключение сессии, чтобы можно было сверяться.

При этом, исходя из материала про SegmentStream, из двух заданий данное задание является ключевым: Ваш подход использует в том числе поведеченские данные (всевозможные хиты) сессии для определения вклада канала в целевую конверсию - предсказывается вероятность совершить ее в начале и в конце сессии, далее берется разница, и именно эта дельта и определяет вклад канала/кампании. Поэтому чем точнее это будет сделано, тем лучше будут взвешены сами сессии. Если что-то будет отклонятся от правил определения сессии, хиты будут распределяться неверно, а значит и оценка каждой сессии будет ошибочной.

Правило про utm_campaign оказался наиболее спорным. Справка указывает:
- If a user arrives via one campaign, leaves, and then comes back via a different campaign.
- Every time a user's campaign source changes, Analytics opens a new session. It’s important to point out that even if an existing session is still open (that is, less than 30 mins have elapsed), if the campaign source changes mid-session the first session is closed and a new session is opened.
- Analytics stores campaign source information. Each time the value of the campaign is updated, Analytics opens a new session

AnonymousId = '0ef42050-0c05-11e8-bcd2-9b2327f4809e', самый, как мне кажется, показательный в этом плане (запрос представлен после задания). У него самое большое количество сессий, рассчитанное по запросу. Ниже сразу после вывода подсчета всех сессий представлены данные конкретно по этому юзеру. Видно, что у него то была кампания, то Null (переписал на None), потом снова тот же id, потом другой. Возможно, тот же запись с тем же id кампании могла иметь другой search term:
- Each search term updates the campaign, so each keyword corresponds to a new session.

Это, по сути, четвертое условие начала новой сессии, но в таблице нет данных о utm_term, так что учесть это не представлялось возможным напрямую, только косвенным образом.

Другой вопрос, что смена происходила по времени хита довольно мгновенно: полсекунды назад был с одной кампании, а после - с другой. А потом снова с первой и пр. Часто кампании не было совсем. Что говорит предположительно о том, что хиты некорректно отправляются (например, проблемы с установкой тэга). Если так, то проведя обработку Null (None) заменой на предыдущее ненулевое значение, мы легко сможем это исправить через COALESCE.
Условно: COALESCE(old_camp, LAST_VALUE(old_camp IGNORE NULLS) OVER (PARTITION BY anonymousId ORDER BY hitTime)) 

***
В ночь с воскресения на понедельник понял, что эту функцию можно успешно внедрить для индентификатора сессии и убрать колонку session_start, сделав запрос и таблицу более удобной для использования - становится лучше видно, какие хиты к какой сессии относятся. Еще один запрос будет чуть ниже на примере того же anonymousId = '0ef42050-0c05-11e8-bcd2-9b2327f4809e'
***

Также стоит для прояснения деталей копнуть в справку. 

Понимая важность размежевания сессий, я написал к таблице еще два запроса, но через GROUP BY, чтобы посмотреть, сколько они дают сессий. Один запрос давал 705, другой - 799 сессий: оба реализовывались через по схожей между собой логике, первый учитывал разницу с микросекундах и датах, другой разницу еще между текущей и предыдущей кампаний, что в первом случае сглаживало переходы из кампании, заход с которых ранее уже был, но, строго говоря, это нарушало правила новой сессии по кампании (идентичные строки схлопывались). Второй переходы учитывал, поэтому на уникального юзера строк вышло больше.

Я сравнил average sessions per user двух запросов по данным из открытых источников:
Сначала взял отсюда, так как данные у нас в таблице за 2018 год
- https://moz.com/blog/ecommerce-benchmark-kpi-study-2018
Потом отсюда (2020)
- https://www.littledata.io/average/sessions-per-user 

Среднее в ~1.5

У опубликованного запроса avg = 1.35 vs 1.29 у первого альтернативного vs 2.14 у второго альтернативного, что делает опубликованный более убедительным в плане правильности подсчета сессий (находится ближе к среднему по результатам исследований).

Альтернативные запросы здесь не приводил, но могу добавить, если необходимо.

Допускаю, что не попал в конкретную цифру, но, надеюсь, логика рассуждения подсчета сессий смогла хоть как-то это скомпенсировать :)

In [ ]:
query = '''
--переменная для того, чтобы "30" не было магическим числом
DECLARE thirtyMinutes INT64 DEFAULT 30;

WITH stats AS 
    (SELECT 
        * EXCEPT(hitTime),
        SUM(session_start) OVER (PARTITION BY anonymousId) AS total_user_sessions
    FROM
        (SELECT 
            anonymousId,
            utm_campaign,
            hitTime,
            curDate,
            minutes,
            CASE
                WHEN LAG(curDate) OVER (PARTITION BY anonymousId ORDER BY hitTime) IS NULL THEN 1
                WHEN utm_campaign != LAG(utm_campaign) OVER (PARTITION BY anonymousId ORDER BY hitTime) AND LAG(utm_campaign) OVER (PARTITION BY anonymousId ORDER BY hitTime) IS NOT NULL THEN 1
                WHEN curDate != LAG(curDate) OVER (PARTITION BY anonymousId ORDER BY hitTime) AND LAG(curDate) OVER (PARTITION BY anonymousId ORDER BY hitTime) IS NOT NULL THEN 1
                WHEN minutes >= thirtyMinutes THEN 1
                ELSE 0
            END AS session_start 
        FROM
            (SELECT  
                anonymousId,
                IF(utm_campaign IS NULL, 'None', utm_campaign) AS utm_campaign,
                hitTime,
                DATE(hitTime) AS curDate,
                IF(TIMESTAMP_DIFF(hitTime, 
                        LAG(hitTime) OVER (PARTITION BY anonymousId ORDER BY hitTime), 
                        MINUTE) IS NULL, 0,
                TIMESTAMP_DIFF(hitTime, 
                        LAG(hitTime) OVER (PARTITION BY anonymousId ORDER BY hitTime), 
                        MINUTE))
                        AS minutes
            FROM analytics-230012.Assignments.hits_for_sessions)))

SELECT 
SUM(total_user_sessions) AS total_sessions
FROM
  (SELECT
  DISTINCT
  anonymousId,
  total_user_sessions
FROM stats)
'''

project_id = 'segmentstream-309512'

df = pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, dialect='standard')
df.head()

,total_sessions
0,741


Кейс с AnonymousId = '0ef42050-0c05-11e8-bcd2-9b2327f4809e'
Колонка session_start означает начало очередной сессии.

In [ ]:
query = '''
--переменная для того, чтобы 30 не было магическим числом
DECLARE thirtyMinutes INT64 DEFAULT 30;

WITH stats AS 
    (SELECT 
        *,
        SUM(session_start) OVER (PARTITION BY anonymousId) AS total_sessions
    FROM
        (SELECT 
            anonymousId,
            utm_campaign,
            hitTime,
            curDate,
            minutes,
            CASE
                WHEN LAG(curDate) OVER (PARTITION BY anonymousId ORDER BY hitTime) IS NULL THEN 1
                WHEN utm_campaign != LAG(utm_campaign) OVER (PARTITION BY anonymousId ORDER BY hitTime) AND LAG(utm_campaign) OVER (PARTITION BY anonymousId ORDER BY hitTime) IS NOT NULL THEN 1
                WHEN curDate != LAG(curDate) OVER (PARTITION BY anonymousId ORDER BY hitTime) AND LAG(curDate) OVER (PARTITION BY anonymousId ORDER BY hitTime) IS NOT NULL THEN 1
                WHEN minutes >= thirtyMinutes THEN 1
                --WHEN curDate != LEAD(curDate) OVER (PARTITION BY anonymousId ORDER BY hitTime) THEN 1
                WHEN minutes >= thirtyMinutes THEN 1
                ELSE 0
            END AS session_start 
        FROM
            (SELECT  
                anonymousId,
                IF(utm_campaign IS NULL, 'None', utm_campaign) AS utm_campaign,
                hitTime,
                DATE(hitTime) AS curDate,
                IF(TIMESTAMP_DIFF(hitTime, 
                        LAG(hitTime) OVER (PARTITION BY anonymousId ORDER BY hitTime), 
                        MINUTE) IS NULL, 0,
                TIMESTAMP_DIFF(hitTime, 
                        LAG(hitTime) OVER (PARTITION BY anonymousId ORDER BY hitTime), 
                        MINUTE))
                        AS minutes
            FROM analytics-230012.Assignments.hits_for_sessions)))

SELECT 
*
FROM stats
WHERE anonymousId = '0ef42050-0c05-11e8-bcd2-9b2327f4809e'
'''
project_id = 'segmentstream-309512'

df = pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, dialect='standard')
df

,anonymousId,utm_campaign,hitTime,curDate,minutes,session_start,total_sessions
0,0ef42050-0c05-11e8-bcd2-9b2327f4809e,a9Yj/+1ZvP9IJ1OM0mHmfg==,2018-02-07 12:47:31.417999+00:00,2018-02-07,0,1,11
1,0ef42050-0c05-11e8-bcd2-9b2327f4809e,a9Yj/+1ZvP9IJ1OM0mHmfg==,2018-02-07 12:47:31.486999+00:00,2018-02-07,0,0,11
2,0ef42050-0c05-11e8-bcd2-9b2327f4809e,a9Yj/+1ZvP9IJ1OM0mHmfg==,2018-02-07 12:47:34.684999+00:00,2018-02-07,0,0,11
3,0ef42050-0c05-11e8-bcd2-9b2327f4809e,a9Yj/+1ZvP9IJ1OM0mHmfg==,2018-02-07 12:47:42.233000+00:00,2018-02-07,0,0,11
4,0ef42050-0c05-11e8-bcd2-9b2327f4809e,a9Yj/+1ZvP9IJ1OM0mHmfg==,2018-02-07 12:47:44.292999+00:00,2018-02-07,0,0,11
5,0ef42050-0c05-11e8-bcd2-9b2327f4809e,None,2018-02-07 12:47:47.584000+00:00,2018-02-07,0,1,11
6,0ef42050-0c05-11e8-bcd2-9b2327f4809e,None,2018-02-07 12:47:47.589999+00:00,2018-02-07,0,0,11
7,0ef42050-0c05-11e8-bcd2-9b2327f4809e,None,2018-02-07 12:48:30.641999+00:00,2018-02-07,0,0,11
8,0ef42050-0c05-11e8-bcd2-9b2327f4809e,None,2018-02-07 12:48:30.642999+00:00,2018-02-07,0,0,11
9,0ef42050-0c05-11e8-bcd2-9b2327f4809e,None,2018-02-07 12:51:01.130000+00:00,2018-02-07,2,0,11


Индентификатор сессии на примере того же anonymousId = '0ef42050-0c05-11e8-bcd2-9b2327f4809e'

- Комбинирование anonymousId, utm_campaign, hitTime

Как оказалось, одного hitTime было недостаточно, так как попался юзер = 'e82da240-0c4d-11e8-adae-b136c2ccc25d', у которого два хита были в одно и тоже время, но с разных кампаний, поэтому общее число сессий было равно 740.

In [ ]:
query = '''
--переменная для того, чтобы 30 не было магическим числом
DECLARE thirtyMinutes INT64 DEFAULT 30;

WITH stats AS 
    (SELECT
        * EXCEPT(minutes, session_start, sessionId),
        COALESCE(sessionId, LAST_VALUE(sessionId IGNORE NULLS) OVER (PARTITION BY anonymousId ORDER BY hitTime)) AS sessionId
    FROM
        (SELECT 
            anonymousId,
            utm_campaign,
            hitTime,
            curDate,
            minutes,
            session_start,
            IF(session_start = 1, CONCAT('session', '_', anonymousId, '_', CAST(hitTime AS STRING), '_', utm_campaign), NULL) AS sessionId
        FROM
            (SELECT 
                anonymousId,
                old_camp,
                utm_campaign,
                hitTime,
                curDate,
                minutes,
                CASE
                    WHEN LAG(curDate) OVER (PARTITION BY anonymousId ORDER BY hitTime) IS NULL THEN 1
                    WHEN utm_campaign != LAG(utm_campaign) OVER (PARTITION BY anonymousId ORDER BY hitTime) AND LAG(utm_campaign) OVER (PARTITION BY anonymousId ORDER BY hitTime) IS NOT NULL THEN 1
                    WHEN curDate != LAG(curDate) OVER (PARTITION BY anonymousId ORDER BY hitTime) AND LAG(curDate) OVER (PARTITION BY anonymousId ORDER BY hitTime) IS NOT NULL THEN 1
                    WHEN minutes >= thirtyMinutes THEN 1
                    --WHEN curDate != LEAD(curDate) OVER (PARTITION BY anonymousId ORDER BY hitTime) THEN 1
                    WHEN minutes >= thirtyMinutes THEN 1
                    ELSE 0
                END AS session_start 
            FROM
                (SELECT  
                    anonymousId,
                    utm_campaign AS old_camp,
                    IF(utm_campaign IS NULL, 'None', utm_campaign) AS utm_campaign,
                    hitTime,
                    DATE(hitTime) AS curDate,
                    IF(TIMESTAMP_DIFF(hitTime, 
                            LAG(hitTime) OVER (PARTITION BY anonymousId ORDER BY hitTime), 
                            MINUTE) IS NULL, 0,
                    TIMESTAMP_DIFF(hitTime, 
                            LAG(hitTime) OVER (PARTITION BY anonymousId ORDER BY hitTime), 
                            MINUTE))
                            AS minutes
                FROM analytics-230012.Assignments.hits_for_sessions))))

SELECT
    *
FROM stats
WHERE anonymousId = '0ef42050-0c05-11e8-bcd2-9b2327f4809e'
'''
project_id = 'segmentstream-309512'

df = pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, dialect='standard')
df

,anonymousId,utm_campaign,hitTime,curDate,sessionId
0,0ef42050-0c05-11e8-bcd2-9b2327f4809e,a9Yj/+1ZvP9IJ1OM0mHmfg==,2018-02-07 12:47:31.417999+00:00,2018-02-07,session_0ef42050-0c05-11e8-bcd2-9b2327f4809e_2...
1,0ef42050-0c05-11e8-bcd2-9b2327f4809e,a9Yj/+1ZvP9IJ1OM0mHmfg==,2018-02-07 12:47:31.486999+00:00,2018-02-07,session_0ef42050-0c05-11e8-bcd2-9b2327f4809e_2...
2,0ef42050-0c05-11e8-bcd2-9b2327f4809e,a9Yj/+1ZvP9IJ1OM0mHmfg==,2018-02-07 12:47:34.684999+00:00,2018-02-07,session_0ef42050-0c05-11e8-bcd2-9b2327f4809e_2...
3,0ef42050-0c05-11e8-bcd2-9b2327f4809e,a9Yj/+1ZvP9IJ1OM0mHmfg==,2018-02-07 12:47:42.233000+00:00,2018-02-07,session_0ef42050-0c05-11e8-bcd2-9b2327f4809e_2...
4,0ef42050-0c05-11e8-bcd2-9b2327f4809e,a9Yj/+1ZvP9IJ1OM0mHmfg==,2018-02-07 12:47:44.292999+00:00,2018-02-07,session_0ef42050-0c05-11e8-bcd2-9b2327f4809e_2...
5,0ef42050-0c05-11e8-bcd2-9b2327f4809e,None,2018-02-07 12:47:47.584000+00:00,2018-02-07,session_0ef42050-0c05-11e8-bcd2-9b2327f4809e_2...
6,0ef42050-0c05-11e8-bcd2-9b2327f4809e,None,2018-02-07 12:47:47.589999+00:00,2018-02-07,session_0ef42050-0c05-11e8-bcd2-9b2327f4809e_2...
7,0ef42050-0c05-11e8-bcd2-9b2327f4809e,None,2018-02-07 12:48:30.641999+00:00,2018-02-07,session_0ef42050-0c05-11e8-bcd2-9b2327f4809e_2...
8,0ef42050-0c05-11e8-bcd2-9b2327f4809e,None,2018-02-07 12:48:30.642999+00:00,2018-02-07,session_0ef42050-0c05-11e8-bcd2-9b2327f4809e_2...
9,0ef42050-0c05-11e8-bcd2-9b2327f4809e,None,2018-02-07 12:51:01.130000+00:00,2018-02-07,session_0ef42050-0c05-11e8-bcd2-9b2327f4809e_2...


#Assignment #2
Count all the users matching the following criteria:
- Have multiple transactions
- Have at least one transaction that was made within 7 days from the previous transaction

**In**:
 - Public table with samle transactions data: `analytics-230012.Assignments.orders`
 - Use your Google Cloud project ID instead of  `project_id`

**Комментарии:**


---

  Это задание показалось мне куда как легче, чем первое. Но так как оно шло вторым после довольно непростого первого, кажется, что оно не может быть таким легким, что там были какие-то подводные камни, которые я как ни искал, не нашел :)

  В запросе я использовал условие "Have at least one transaction that was made within 7 days from the previous transaction", потому что из него логически следует, что юзер has multiple transactions.

  Пожалуй, единственный подводный был в том, что надо было только не забыть вычесть из таблицы тех, у кого разница между заказами нулевая (вытекает из оконной Lead), тогда как остальные разницы между заказами в рамках этого юзера могут и не удовлетворять условию < 7 дней.  

In [ ]:
query = '''
#Start your query here
--переменная для того, чтобы 604800 не было магическим числом
--7 дней = 604800 секунд
DECLARE sevenDays INT64 DEFAULT 604800;

SELECT
    COUNT(DISTINCT userid) AS total_users
FROM
    (SELECT 
        userid,
        orderid,
        date,
        IF(TIMESTAMP_DIFF(
                LEAD(date) OVER (PARTITION BY userId ORDER BY date),
                date,
                SECOND) IS NULL, 0,
            TIMESTAMP_DIFF(
                LEAD(date) OVER (PARTITION BY userId ORDER BY date),
                date, 
                SECOND)) AS diff_seconds
    FROM analytics-230012.Assignments.orders)
WHERE diff_seconds > 0 AND diff_seconds < sevenDays

#Finish your query here
'''

project_id = 'segmentstream-309512' #change

df = pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, dialect='standard')
df.head()

,total_users
0,1561


Для проверки (я б сказал, паранойи ради) использовал этот запрос, в котором были доп. колонки:
- дата следующего заказа
- кол-во заказов на юзера
- разница в секундах между этими датами
- больше ли 7 дней следующий заказ (главное, чтоб один раз было True)


In [ ]:
query = '''
--переменная для того, чтобы 604800 не было магическим числом
--7 дней = 604800 секунд
DECLARE sevenDays INT64 DEFAULT 604800;

SELECT 
        *,
        --специально оставил с секундах, чтобы отсчитать 7 дней вполть до секунды,
        UNIX_SECONDS(TIMESTAMP(nextDate)) - UNIX_SECONDS(TIMESTAMP(date)) AS secondsBetweenOrders,
        UNIX_SECONDS(TIMESTAMP(nextDate)) - UNIX_SECONDS(TIMESTAMP(date)) > sevenDays AS moreThanSevenDays
    FROM
        (SELECT 
            userid,
            orderid,
            date,
            IF(LEAD(date) OVER (PARTITION BY userid ORDER BY date) IS NULL, date, LEAD(date) OVER (PARTITION BY userid ORDER BY date)) AS nextDate,
            COUNT(orderid) OVER (PARTITION BY userid) AS totalOrders
        FROM analytics-230012.Assignments.orders)
WHERE userid = '113711' 
#Finish your query here
'''

project_id = 'segmentstream-309512'

df = pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, dialect='standard')
df

,userid,orderid,date,nextDate,totalOrders,secondsBetweenOrders,moreThanSevenDays
0,113711,BB24630,2017-02-16 00:18:51,2017-02-21 12:06:35,5,474464,False
1,113711,BB76606,2017-02-21 12:06:35,2017-03-05 17:51:17,5,1057482,True
2,113711,BB00607,2017-03-05 17:51:17,2017-03-07 12:27:55,5,153398,False
3,113711,BD80947,2017-03-07 12:27:55,2017-04-05 15:08:52,5,2515257,True
4,113711,BD01941,2017-04-05 15:08:52,2017-04-05 15:08:52,5,0,False


Альтернативные, менее, имхо, элегантные (если, конечно, базовый не зафакачен:)) запросы с таким же результатом:

In [ ]:
query = '''
--переменная для того, чтобы 604800 не было магическим числом
--7 дней = 604800 секунд
DECLARE sevenDays INT64 DEFAULT 604800;

SELECT 
    COUNT(DISTINCT userid) AS total_users
FROM
    (SELECT 
        userid,
        --специально оставил с секундах, чтобы отсчитать 7 дней вполть до секунды
        UNIX_SECONDS(TIMESTAMP(nextDate)) - UNIX_SECONDS(TIMESTAMP(date)) AS secondsBetweenOrders
    FROM
        (SELECT 
            userid,
            date,
            IF(LEAD(date) OVER (PARTITION BY userid ORDER BY date) IS NULL, date, LEAD(date) OVER (PARTITION BY userid ORDER BY date)) AS nextDate,
            COUNT(orderid) OVER (PARTITION BY userid) AS totalOrders
        FROM analytics-230012.Assignments.orders)
    WHERE totalOrders > 1)
WHERE secondsBetweenOrders < sevenDays AND secondsBetweenOrders > 0
'''

project_id = 'segmentstream-309512'

df = pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, dialect='standard')
df.head()

,total_users
0,1561


In [ ]:
query = '''
--переменная для того, чтобы 604800 не было магическим числом
--7 дней = 604800 секунд
DECLARE sevenDays INT64 DEFAULT 604800;

WITH stats AS
(SELECT 
    *,
    MIN(secondsBetweenOrders) OVER (PARTITION BY userid) AS minDiff
FROM
    (SELECT 
        userid,
        orderid,
        date,
        nextDate,
        totalOrders,
        --специально оставил с секундах, чтобы отсчитать 7 дней вполть до секунды
        UNIX_SECONDS(TIMESTAMP(nextDate)) - UNIX_SECONDS(TIMESTAMP(date)) AS secondsBetweenOrders
    FROM
        (SELECT 
            userid,
            orderid,
            date,
            IF(LEAD(date) OVER (PARTITION BY userid ORDER BY date) IS NULL, date, LEAD(date) OVER (PARTITION BY userid ORDER BY date)) AS nextDate,
            COUNT(orderid) OVER (PARTITION BY userid) AS totalOrders
        FROM analytics-230012.Assignments.orders)
    WHERE totalOrders > 1)
WHERE secondsBetweenOrders > 0)

SELECT
    COUNT(DISTINCT userid) AS total_users
FROM stats
WHERE minDiff < sevenDays
'''

project_id = 'segmentstream-309512'

df = pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, dialect='standard')
df.head()

,total_users
0,1561
